In [6]:
import os
from PIL import Image
from torchvision import transforms
from tqdm import tqdm  # Progress bar
import torch

# Original and Augmented Data Paths
original_data_dir = "pneumonia/train"
augmented_data_dir = "pneumonia-augmentated/train"


# Copy class-wise folder structure
for class_name in os.listdir(original_data_dir):
    class_path = os.path.join(original_data_dir, class_name)
    if os.path.isdir(class_path):  # Only process directories
        os.makedirs(os.path.join(augmented_data_dir, class_name), exist_ok=True)

# Define Augmentation Transformations
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.RandomApply([transforms.RandomRotation(degrees=10)], p=0.5),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0))], p=0.5),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0)], p=0.5),
    transforms.ToTensor(),
    transforms.RandomApply([transforms.Lambda(lambda x: x + 0.05 * torch.randn_like(x))], p=0.5),
])

# Process Each Image and Save
for class_name in os.listdir(original_data_dir):
    class_path = os.path.join(original_data_dir, class_name)
    augmented_class_path = os.path.join(augmented_data_dir, class_name)
    
    if os.path.isdir(class_path):  # Ensure it's a directory
        for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
            img_path = os.path.join(class_path, img_name)
            image = Image.open(img_path).convert("RGB")  # Open image
            
            augmented_image = transform(image)  # Apply transformations
            augmented_image = transforms.ToPILImage()(augmented_image)  # Convert back to PIL
            
            # Save the new image
            new_img_name = f"aug_{img_name}"
            augmented_image.save(os.path.join(augmented_class_path, new_img_name))

print("Augmented images saved successfully!")


Processing NORMAL: 100%|██████████| 1341/1341 [00:23<00:00, 55.90it/s]

Augmented images saved successfully!
